In [1]:
# import pandas to read in CSV files
import pandas as pd

# import matplotlib + seaborn for exploratory data analysis
import plotly.graph_objects as go

In [2]:
# read in accident data CSVs for 2015 - 2017
acc15 = pd.read_csv('/Users/ianforrest/Desktop/coding/repos/DS_Safe-Routes/modeling/accident15.csv')
acc16 = pd.read_csv('/Users/ianforrest/Desktop/coding/repos/DS_Safe-Routes/modeling/accident16.csv')
acc17 = pd.read_csv('/Users/ianforrest/Desktop/coding/repos/DS_Safe-Routes/modeling/accident17.csv')

In [3]:
# combine acc15, acc16, acc17 into one dataframe
acc = acc15.append(acc16, sort=False, ignore_index = True)
acc = acc.append(acc17, sort=False, ignore_index = True)

In [4]:
acc.shape

(101533, 52)

In [5]:
# display columns of combined acc dataframe
acc.columns

Index(['STATE', 'ST_CASE', 'VE_TOTAL', 'VE_FORMS', 'PVH_INVL', 'PEDS',
       'PERNOTMVIT', 'PERMVIT', 'PERSONS', 'COUNTY', 'CITY', 'DAY', 'MONTH',
       'YEAR', 'DAY_WEEK', 'HOUR', 'MINUTE', 'NHS', 'RUR_URB', 'FUNC_SYS',
       'RD_OWNER', 'ROUTE', 'TWAY_ID', 'TWAY_ID2', 'MILEPT', 'LATITUDE',
       'LONGITUD', 'SP_JUR', 'HARM_EV', 'MAN_COLL', 'RELJCT1', 'RELJCT2',
       'TYP_INT', 'WRK_ZONE', 'REL_ROAD', 'LGT_COND', 'WEATHER1', 'WEATHER2',
       'WEATHER', 'SCH_BUS', 'RAIL', 'NOT_HOUR', 'NOT_MIN', 'ARR_HOUR',
       'ARR_MIN', 'HOSP_HR', 'HOSP_MN', 'CF1', 'CF2', 'CF3', 'FATALS',
       'DRUNK_DR'],
      dtype='object')

In [30]:
# create city/state/latitude/longitude dataframe
acc_city_state = acc[['CITY', 'STATE', 'LATITUDE', 'LONGITUD']]

# remove unknown cities
acc_city_state = acc_city_state[acc_city_state['CITY'] > 0]
acc_city_state = acc_city_state[acc_city_state['CITY'] < 9997]

# remove unknown latitude/longitude
# acc_city_state = acc_city_state[acc_city_state['LATITUDE'] < 700]
# acc_city_state = acc_city_state[acc_city_state['LONGITUD'] < 700]

# add 'COUNT' column to dataframe, counts number of CITY/STATE combinations
acc_city_state['COUNT'] = acc_city_state.groupby(['CITY', 'STATE'])['STATE'].transform('count')
acc_city_state = acc_city_state.sort_values(by=['COUNT'], ascending=False)
acc_city_state = acc_city_state.groupby('COUNT').first().sort_values(by=['COUNT'], ascending=False)
acc_city_state = acc_city_state.reset_index()
top_100 = acc_city_state.head(100)

top_100 = top_100.replace(4120, 3382)
top_100.head(37)

,COUNT,CITY,STATE,LATITUDE,LONGITUD
0,788,1980,6,34.221669,-118.413881
1,668,3280,48,29.666778,-95.552475
2,652,4170,36,40.509711,-74.247503
3,623,370,4,33.375819,-112.168453
4,512,1730,48,32.768761,-96.822747
5,471,6090,48,29.396403,-98.479578
6,398,1510,12,30.253192,-81.699192
7,364,1670,17,41.764706,-87.683264
8,321,1260,26,42.434400,-83.006897
9,307,1620,47,35.102233,-89.947192


In [31]:
# City/State dictionary to interpret city/state geocodes
df = pd.read_csv('/Users/ianforrest/Desktop/coding/repos/DS_Safe-Routes/visualizations/FRPP GLC United States (1).xlsx - GeoLocation_UnitedStates.csv')
df.columns = ['ST_NAME', 'STATE', 'CITY', 'CT_NAME']
df.head()

,ST_NAME,STATE,CITY,CT_NAME
0,ALABAMA,1,220,AUTAUGAVILLE
1,ALABAMA,1,340,BILLINGSLEY
2,ALABAMA,1,425,BOOTH
3,ALABAMA,1,1796,JONES
4,ALABAMA,1,2002,MARBURY


In [32]:
# limit dataframe to top 100 cities with most crashes
top_100 = top_100.merge(df, how='left').drop_duplicates(['CITY', 'STATE'])

# format text of city/state columns
top_100['ST_NAME'] = top_100['ST_NAME'].str.title()
top_100['CT_NAME'] = top_100['CT_NAME'].str.title()

# combine city name, state name, and number of accidents into 'DESC' column
top_100['DESC'] = top_100['CT_NAME'] + ', ' + top_100['ST_NAME'] + ' - ' + top_100['COUNT'].map(str) + ' Accidents'

In [34]:
# check 'DESC' column added successfully
top_100.head(100)

,COUNT,CITY,STATE,LATITUDE,LONGITUD,ST_NAME,CT_NAME,DESC
0,788,1980,6,34.221669,-118.413881,California,Los Angeles,"Los Angeles, California - 788 Accidents"
1,668,3280,48,29.666778,-95.552475,Texas,Houston,"Houston, Texas - 668 Accidents"
4,652,4170,36,40.509711,-74.247503,New York,New York City,"New York City, New York - 652 Accidents"
9,623,370,4,33.375819,-112.168453,Arizona,Phoenix,"Phoenix, Arizona - 623 Accidents"
10,512,1730,48,32.768761,-96.822747,Texas,Dallas,"Dallas, Texas - 512 Accidents"
15,471,6090,48,29.396403,-98.479578,Texas,San Antonio,"San Antonio, Texas - 471 Accidents"
16,398,1510,12,30.253192,-81.699192,Florida,Jacksonville,"Jacksonville, Florida - 398 Accidents"
17,364,1670,17,41.764706,-87.683264,Illinois,Chicago,"Chicago, Illinois - 364 Accidents"
19,321,1260,26,42.434400,-83.006897,Michigan,Detroit,"Detroit, Michigan - 321 Accidents"
20,307,1620,47,35.102233,-89.947192,Tennessee,Memphis,"Memphis, Tennessee - 307 Accidents"


In [18]:
# create Plotly Scattergeo plot for top 100 accident cities from 2015 - 2017
fig = go.Figure(data=go.Scattergeo(
        lon = top_100['LONGITUD'],
        lat = top_100['LATITUDE'],
        text = top_100['DESC'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.5,
            reversescale = True,
            autocolorscale = True,
            symbol = 'circle'),
        marker_color = '#13F1FC',
        hoverinfo = 'text',
        ))

fig.update_layout(
        geo_scope='usa',
    )
fig.show()